In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator 
from keras.utils import to_categorical
from keras.models import Sequential
from keras.models import model_from_json
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

Using TensorFlow backend.


In [2]:
sl_train=pd.read_csv('sign_mnist_train.csv')
sl_test=pd.read_csv('sign_mnist_test.csv')

X_train=np.array(sl_train.iloc[:,1:])
y_train=to_categorical(np.array(sl_train.iloc[:,0]))
X_test=np.array(sl_test.iloc[:,1:])
y_test=to_categorical(np.array(sl_test.iloc[:,0]))

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_train = X_train.astype('float32')
#X_train /= 255

X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_test = X_test.astype('float32')
X_test /= 255

In [3]:
datagen=ImageDataGenerator(rotation_range=30,zoom_range=0.2,rescale=1./255)

In [4]:
test_sb=pd.read_csv('shrinkthenblackalldata.csv')
test_bs=pd.read_csv('blackthenshrinkALLDATA.csv')

In [5]:
x_test_sb=test_sb.drop(columns=['file name','expected output','expected output num'])
x_test_bs=test_bs.drop(columns=['file name','expected output','expected output num'])

y_test_sb=to_categorical(np.array(test_sb['expected output num'].iloc[:]))
y_test_bs=to_categorical(np.array(test_bs['expected output num'].iloc[:]))

x_test_sb=np.array(x_test_sb.iloc[:,:])
x_test_bs=np.array(x_test_bs.iloc[:,:])

x_test_sb = x_test_sb.reshape(x_test_sb.shape[0], 28, 28, 1)
x_test_sb = x_test_sb.astype('float32')
x_test_sb /= 255

x_test_bs = x_test_bs.reshape(x_test_bs.shape[0], 28, 28, 1)
x_test_bs = x_test_bs.astype('float32')
x_test_bs /= 255

In [6]:
cnn=Sequential()

cnn.add(Conv2D(32,kernel_size=(3,3),activation='relu',
               input_shape=(28,28,1),padding='same'))
cnn.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
cnn.add(BatchNormalization())
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Flatten())

cnn.add(Dense(256,activation='relu'))
cnn.add(BatchNormalization())
cnn.add(Dropout(0.5))

cnn.add(Dense(25,activation='softmax'))
cnn.compile(loss='categorical_crossentropy',optimizer='adam',
            metrics=['accuracy'])
cnn.fit_generator(datagen.flow(X_train,y_train,batch_size=32),
                  validation_data=(x_test_bs,y_test_bs),epochs=10)

Epoch 1/10
858/858 [==============================] - 43s 50ms/step - loss: 1.1449 - accuracy: 0.6528 - val_loss: 4.1330 - val_accuracy: 0.1538
Epoch 2/10
858/858 [==============================] - 42s 49ms/step - loss: 0.4097 - accuracy: 0.8643 - val_loss: 5.3544 - val_accuracy: 0.1731
Epoch 3/10
858/858 [==============================] - 48s 56ms/step - loss: 0.2816 - accuracy: 0.9058 - val_loss: 7.4286 - val_accuracy: 0.1474
Epoch 4/10
858/858 [==============================] - 45s 53ms/step - loss: 0.2076 - accuracy: 0.9321 - val_loss: 6.1968 - val_accuracy: 0.1603
Epoch 5/10
858/858 [==============================] - 47s 54ms/step - loss: 0.1753 - accuracy: 0.9412 - val_loss: 5.5555 - val_accuracy: 0.2051
Epoch 6/10
858/858 [==============================] - 47s 55ms/step - loss: 0.1510 - accuracy: 0.9487 - val_loss: 4.7799 - val_accuracy: 0.1538
Epoch 7/10
858/858 [==============================] - 48s 56ms/step - loss: 0.1355 - accuracy: 0.9544 - val_loss: 5.5723 - val_accuracy:

In [7]:
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 26, 26, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)              

In [8]:
model_json = cnn.to_json()
with open("sl_model.json", "w") as json_file:
    json_file.write(model_json)
cnn.save_weights("sl_model.h5")